LDA_w2vfor3levels

In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [2]:
import warnings
warnings.filterwarnings("ignore") 

In [3]:
import os 
import numpy as np 
import pandas as pd
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

In [4]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/LDA_on_embeddings/data.csv"
df = pd.read_csv(path)
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [5]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [6]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [7]:
#define bins
bins = [0,2,4,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names) 

In [8]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [9]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [10]:
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [11]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/LDA_on_embeddings/w2v_2lda_withstopwordchangelevels_2_features.csv"
x = pd.read_csv(path)
x.head()

,Unnamed: 0,0,1
0,0,1.656071,-0.583930
1,1,-0.417732,1.481841
2,2,1.336153,0.029915
3,3,-0.130531,-0.063801
4,4,-0.391408,0.738711


In [12]:
x.drop(x.columns[[0]], axis=1, inplace=True)
x.head()

,0,1
0,1.656071,-0.583930
1,-0.417732,1.481841
2,1.336153,0.029915
3,-0.130531,-0.063801
4,-0.391408,0.738711


In [13]:
y=df['Level '] 

**Train Test Split**

In [14]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 2), (524, 2))

In [15]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

2    45.327869
1    29.590164
3    25.081967
Name: Level , dtype: float64


2    45.229008
1    29.961832
3    24.809160
Name: Level , dtype: float64


**Model Building**

In [ ]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [ ]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test) 

Perceptron F1 score: 0.4360009556982113


**Decision Tree**

In [16]:
clf_dt = DecisionTreeClassifier(max_depth=5, class_weight = "balanced")

In [17]:
clf_dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [18]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [19]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [20]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [21]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.406557,0.406557,0.457436,0.404198,0.316794,0.316794,0.341352,0.310771


**Random Forest**

In [22]:
clf_rf = RandomForestClassifier(class_weight = "balanced")

In [23]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [24]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [25]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9972    1.0000    0.9986       361
           2     1.0000    1.0000    1.0000       553
           3     1.0000    0.9967    0.9984       306

    accuracy                         0.9992      1220
   macro avg     0.9991    0.9989    0.9990      1220
weighted avg     0.9992    0.9992    0.9992      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3008    0.2548    0.2759       157
           2     0.4477    0.5232    0.4825       237
           3     0.2632    0.2308    0.2459       130

    accuracy                         0.3702       524
   macro avg     0.3372    0.3363    0.3348       524
weighted avg     0.3579    0.3702    0.3619       524



In [26]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.406557,0.406557,0.457436,0.404198,0.316794,0.316794,0.341352,0.310771
1,RandomForest,0.999180,0.999180,0.999183,0.999180,0.370229,0.370229,0.357867,0.361885


**AdaBoost**

In [ ]:
clf_adaboost =  AdaBoostClassifier()

In [ ]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4536    0.1219    0.1921       361
           2     0.4817    0.9259    0.6337       553
           3     0.5167    0.1013    0.1694       306

    accuracy                         0.4811      1220
   macro avg     0.4840    0.3830    0.3317      1220
weighted avg     0.4821    0.4811    0.3866      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2830    0.0955    0.1429       157
           2     0.4447    0.8143    0.5753       237
           3     0.2162    0.0615    0.0958       130

    accuracy                         0.4122       524
   macro avg     0.3146    0.3238    0.2713       524
weighted avg     0.3396    0.4122    0.3268       524



In [ ]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.483607,0.483607,0.592522,0.368669,0.433206,0.433206,0.344208,0.297634
1,RandomForest,0.999180,0.999180,0.999183,0.999180,0.381679,0.381679,0.371549,0.375282
2,Adaboost,0.481148,0.481148,0.482138,0.386569,0.412214,0.412214,0.339573,0.326757


**Gradient Boosting Classifier**

In [ ]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.8380    0.4155    0.5556       361
           2     0.5939    0.9783    0.7391       553
           3     0.9231    0.3922    0.5505       306

    accuracy                         0.6648      1220
   macro avg     0.7850    0.5953    0.6150      1220
weighted avg     0.7487    0.6648    0.6375      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3636    0.2038    0.2612       157
           2     0.4599    0.7257    0.5630       237
           3     0.2903    0.1385    0.1875       130

    accuracy                         0.4237       524
   macro avg     0.3713    0.3560    0.3372       524
weighted avg     0.3890    0.4237    0.3794       524



In [ ]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.483607,0.483607,0.592522,0.368669,0.433206,0.433206,0.344208,0.297634
1,RandomForest,0.999180,0.999180,0.999183,0.999180,0.381679,0.381679,0.371549,0.375282
2,Adaboost,0.481148,0.481148,0.482138,0.386569,0.412214,0.412214,0.339573,0.326757
3,GBM,0.664754,0.664754,0.748669,0.637461,0.423664,0.423664,0.388984,0.379429


**SVM Model**

In [27]:
linear_svm = SVC(kernel='linear', C=1, class_weight  ="balanced")

In [28]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, class_weight='balanced', kernel='linear')

In [29]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [30]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.2957    0.1884    0.2301       361
           2     0.5179    0.2098    0.2986       553
           3     0.2546    0.6373    0.3638       306

    accuracy                         0.3107      1220
   macro avg     0.3560    0.3451    0.2975      1220
weighted avg     0.3861    0.3107    0.2947      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2326    0.1274    0.1646       157
           2     0.3918    0.1603    0.2275       237
           3     0.2522    0.6615    0.3652       130

    accuracy                         0.2748       524
   macro avg     0.2922    0.3164    0.2524       524
weighted avg     0.3094    0.2748    0.2428       524



In [31]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.406557,0.406557,0.457436,0.404198,0.316794,0.316794,0.341352,0.310771
1,RandomForest,0.999180,0.999180,0.999183,0.999180,0.370229,0.370229,0.357867,0.361885
2,SVM,0.310656,0.310656,0.386069,0.294684,0.274809,0.274809,0.309433,0.242834


**KNN**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4781    0.5152    0.4960       361
           2     0.5859    0.7342    0.6517       553
           3     0.5145    0.2320    0.3198       306

    accuracy                         0.5434      1220
   macro avg     0.5262    0.4938    0.4892      1220
weighted avg     0.5361    0.5434    0.5224      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3239    0.3631    0.3423       157
           2     0.4397    0.5232    0.4778       237
           3     0.2424    0.1231    0.1633       130

    accuracy                         0.3760       524
   macro avg     0.3353    0.3364    0.3278       524
weighted avg     0.3561    0.3760    0.3592       524



In [ ]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.483607,0.483607,0.592522,0.368669,0.433206,0.433206,0.344208,0.297634
1,RandomForest,0.999180,0.999180,0.999183,0.999180,0.381679,0.381679,0.371549,0.375282
2,Adaboost,0.481148,0.481148,0.482138,0.386569,0.412214,0.412214,0.339573,0.326757
3,GBM,0.664754,0.664754,0.748669,0.637461,0.423664,0.423664,0.388984,0.379429
4,SVM,0.453279,0.453279,0.205462,0.282756,0.452290,0.452290,0.204566,0.281716
5,KNN,0.543443,0.543443,0.536087,0.522379,0.375954,0.375954,0.356058,0.359200


**GNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [ ]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.3226    0.0277    0.0510       361
           2     0.4509    0.9638    0.6144       553
           3     0.0000    0.0000    0.0000       306

    accuracy                         0.4451      1220
   macro avg     0.2578    0.3305    0.2218      1220
weighted avg     0.2998    0.4451    0.2936      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2500    0.0255    0.0462       157
           2     0.4508    0.9662    0.6148       237
           3     0.0000    0.0000    0.0000       130

    accuracy                         0.4447       524
   macro avg     0.2336    0.3306    0.2203       524
weighted avg     0.2788    0.4447    0.2919       524



In [ ]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores) 
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.483607,0.483607,0.592522,0.368669,0.433206,0.433206,0.344208,0.297634
1,RandomForest,0.999180,0.999180,0.999183,0.999180,0.381679,0.381679,0.371549,0.375282
2,Adaboost,0.481148,0.481148,0.482138,0.386569,0.412214,0.412214,0.339573,0.326757
3,GBM,0.664754,0.664754,0.748669,0.637461,0.423664,0.423664,0.388984,0.379429
4,SVM,0.453279,0.453279,0.205462,0.282756,0.452290,0.452290,0.204566,0.281716
5,KNN,0.543443,0.543443,0.536087,0.522379,0.375954,0.375954,0.356058,0.359200
6,GNB,0.445082,0.445082,0.299849,0.293596,0.444656,0.444656,0.278791,0.291907
